In [1]:
import pandas as pd

# Load each dataset
df_demo = pd.read_csv('df_final_demo.txt', sep=',')
df_experiment = pd.read_csv('df_final_experiment_clients.txt', sep=',')
df_web_1 = pd.read_csv('df_final_web_data_pt_1.txt', sep=',')
df_web_2 = pd.read_csv('df_final_web_data_pt_2.txt', sep=',')

# Combine web data
df_web = pd.concat([df_web_1, df_web_2], ignore_index=True)

# Display sample of each dataframe
print("Sample of df_demo:")
print(df_demo.head(), '\n')

print("Sample of df_experiment:")
print(df_experiment.head(), '\n')

print("Sample of df_web:")
print(df_web.head())

Sample of df_demo:
   client_id  clnt_tenure_yr  clnt_tenure_mnth  clnt_age gendr  num_accts  \
0     836976             6.0              73.0      60.5     U        2.0   
1    2304905             7.0              94.0      58.0     U        2.0   
2    1439522             5.0              64.0      32.0     U        2.0   
3    1562045            16.0             198.0      49.0     M        2.0   
4    5126305            12.0             145.0      33.0     F        2.0   

         bal  calls_6_mnth  logons_6_mnth  
0   45105.30           6.0            9.0  
1  110860.30           6.0            9.0  
2   52467.79           6.0            9.0  
3   67454.65           3.0            6.0  
4  103671.75           0.0            3.0   

Sample of df_experiment:
   client_id Variation
0    9988021      Test
1    8320017      Test
2    4033851   Control
3    1982004      Test
4    9294070   Control 

Sample of df_web:
   client_id            visitor_id                      visit_id proc

In [3]:
# Merge web interaction data with experiment variation info
web_exp = pd.merge(df_web, df_experiment, on='client_id', how='inner')

In [5]:
# Filter users who reached the final step 'confirm'
completed = web_exp[web_exp['process_step'] == 'confirm'][['client_id', 'Variation']].drop_duplicates()

In [7]:
# Total unique users in each group
total_users = web_exp[['client_id', 'Variation']].drop_duplicates()
total_counts = total_users['Variation'].value_counts()

# Total completed per group
completion_counts = completed['Variation'].value_counts()

In [10]:
from statsmodels.stats.proportion import proportions_ztest

# Reorder so index matches group
count = [completion_counts['Test'], completion_counts['Control']]
nobs = [total_counts['Test'], total_counts['Control']]

# Perform one-tailed z-test
stat, pval = proportions_ztest(count, nobs, alternative='larger')

print("Z-statistic:", stat)
print("p-value:", pval)

Z-statistic: 8.8745141890702
p-value: 3.511966623790716e-19


In [12]:
# Step 1: Calculate completion rate per group
rate_test = (completion_counts['Test'] / total_counts['Test']) * 100
rate_control = (completion_counts['Control'] / total_counts['Control']) * 100

# Step 2: Calculate difference
diff = rate_test - rate_control

# Step 3: Compare with threshold
threshold = 5

print("Completion Rate (Test):", rate_test)
print("Completion Rate (Control):", rate_control)
print("Difference:", diff)
if diff >= threshold:
    print("✅ Improvement exceeds threshold. New design is cost-effective.")
else:
    print("❌ Improvement does not meet 5% threshold. Not cost-effective.")

Completion Rate (Test): 69.29323642835953
Completion Rate (Control): 65.58728539860616
Difference: 3.705951029753379
❌ Improvement does not meet 5% threshold. Not cost-effective.


In [14]:
# Merge demographics with experiment group info
df_demo_exp = pd.merge(df_demo, df_experiment, on='client_id', how='inner')

# Drop rows with missing age
df_demo_exp = df_demo_exp.dropna(subset=['clnt_age'])

In [16]:
# Split into Test and Control groups
age_test = df_demo_exp[df_demo_exp['Variation'] == 'Test']['clnt_age']
age_control = df_demo_exp[df_demo_exp['Variation'] == 'Control']['clnt_age']

In [18]:
from scipy.stats import ttest_ind

# Perform independent two-sample t-test
stat, pval = ttest_ind(age_test, age_control, equal_var=False)

print("T-statistic:", stat)
print("P-value:", pval)

T-statistic: -2.416068061200627
P-value: 0.015692719461388675


In [20]:
# Merge demographic data with experiment group
df_demo_exp = pd.merge(df_demo, df_experiment, on='client_id', how='inner')

# Drop rows with missing tenure
df_demo_exp = df_demo_exp.dropna(subset=['clnt_tenure_yr'])

# Split into Test and Control groups
tenure_test = df_demo_exp[df_demo_exp['Variation'] == 'Test']['clnt_tenure_yr']
tenure_control = df_demo_exp[df_demo_exp['Variation'] == 'Control']['clnt_tenure_yr']

# Perform independent two-sample t-test
from scipy.stats import ttest_ind

stat, pval = ttest_ind(tenure_test, tenure_control, equal_var=False)

# Show results
print("T-statistic:", stat)
print("P-value:", pval)

T-statistic: -1.7115258395121253
P-value: 0.08699034968593346


In [22]:
# Step 1: Merge demographic and experiment data
df_demo_exp = pd.merge(df_demo, df_experiment, on='client_id', how='inner')

# Step 2: Filter only rows with known gender (exclude 'U')
df_demo_exp = df_demo_exp[df_demo_exp['gendr'].isin(['M', 'F'])]

# Step 3: Create binary variable for gender (e.g., M = 1, F = 0)
df_demo_exp['is_male'] = (df_demo_exp['gendr'] == 'M').astype(int)

# Step 4: Calculate counts for each group
male_test = df_demo_exp[df_demo_exp['Variation'] == 'Test']['is_male'].sum()
male_control = df_demo_exp[df_demo_exp['Variation'] == 'Control']['is_male'].sum()

n_test = df_demo_exp[df_demo_exp['Variation'] == 'Test']['is_male'].count()
n_control = df_demo_exp[df_demo_exp['Variation'] == 'Control']['is_male'].count()

# Step 5: Proportion z-test
from statsmodels.stats.proportion import proportions_ztest

count = [male_test, male_control]
nobs = [n_test, n_control]

stat, pval = proportions_ztest(count, nobs)
print("Z-statistic:", stat)
print("P-value:", pval)

Z-statistic: -1.1615835747534435
P-value: 0.24540465716476256
